In [7]:
import requests
import json
import time
import datetime
from math import *
import sys
import functools
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Retrieve login parameters from config file
with open("config.json") as credentials:
    credentials = json.load(credentials)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]

In [23]:
class JourneyBuilder:

    def __init__(self, client_id, client_secret, journey_name):
        # the function that is executed when
        # an instance of the class is created
        self.client_id = client_id
        self.client_secret = client_secret
        self.journey_name = journey_name
        
        self.auth_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.auth.marketingcloudapis.com/v2/token"
        self.auth_headers = {"content-type": "application/json"}
        self.auth_payload = {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }
        
        self.rest_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.rest.marketingcloudapis.com"

        try:
            self.access_token, self.access_token_expiration, self.rest_headers = self.getAccessToken()
            if self.access_token is None:
                raise Exception("Request for access token failed.")
        except Exception as e:
            print(e)

    def getAccessToken(self):
        # the function that is 
        # used to request the JWT
        try:
            # request an access token
            authentication_response = requests.post(
                url=self.auth_url, data=json.dumps(self.auth_payload), headers=self.auth_headers, verify=False)

            # optional: raise exception for status code
            authentication_response.raise_for_status()
        except Exception as e:
            print(e)
            return None
        else:
            # assuming the response's structure is
            # {"access_token": ""}
            access_token = authentication_response.json()['access_token']
            access_token_expiration = authentication_response.json()['expires_in']
            rest_headers = {"authorization": f"Bearer {access_token}"}
            return access_token, access_token_expiration, rest_headers
    
    class Decorators():
        @staticmethod
        def refreshToken(decorated):
            # Decorator function
            # If func (e.g. pause_journey_version) is passed to @Decorators.refreshToken
            # Then token gets refreshed if expired
            # Wrapper returns func response
            def wrapper(api,*args,**kwargs):
                if time.time() > api.access_token_expiration:
                    api.access_token, api.access_token_expiration, api.rest_headers = api.getAccessToken()
                return decorated(api,*args,**kwargs)

            return wrapper

    @Decorators.refreshToken
    def get_metadata(self, *args):
        # Request gets all the journeys in a dict
        # Get values when name key == "journey_name"
        r = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/",
            headers=self.rest_headers
        )

        return [[journey[arg] for journey in json.loads(r.content)["items"]
                 if journey["name"] == self.journey_name][0] for arg in args]
    
    @Decorators.refreshToken
    def get_activities_by_id(self, journey_id):
        # Request gets all activities within journey based on id
        try:
            r = requests.get(
                url=f'{self.rest_url}/interaction/v1/interactions/{journey_id}',
                headers=self.rest_headers
            )
            return json.loads(r.content)
        except ConnectionError as e:
            print(e)
    
    @Decorators.refreshToken
    def change_status(self, journey_id, journey_version, action):
        # API needs journey id + version to pause
        try:
            r = requests.post(
                url=f"{self.rest_url}/interaction/v1/interactions/{action}/{journey_id}?versionNumber={journey_version}",
                headers=self.rest_headers
            )
        except ConnectionError as e:
            print(e)
            
    @Decorators.refreshToken
    def update_journey_version(self, journey_id, journey_version, new_perc):
        # Pause journey version
        self.change_status(journey_id, journey_version, "pause")
        print(f"Journey: {journey_id, journey_version} PAUSED SUCCESSFULLY!")
        
        # Update journey version
        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc)
        
        try:
            r = requests.put(
                url=f'{self.rest_url}/interaction/v1/interactions',
                headers=self.rest_headers,
                data=json.dumps(new_version_journey_activities)
            )
            print(f"Journey: {journey_id, journey_version} UPDATED SUCCESSFULLY!")
        except ConnectionError as e:
            print(e)
            
        # status needs to be "Paused" and not "Pausing" 
        # API requires some time to do that
        time.sleep(300) 
        
        # Resume journey version
        self.change_status(journey_id, journey_version, "resume")
        print(f"Journey: {journey_id, journey_version} RESUMED SUCCESSFULLY!")
            
    @staticmethod
    def update_journey_perc(journey_activities, new_perc):
        # Update journey version data with new perc
        journey_activities["activities"][0]["outcomes"][0]["arguments"]["percentage"] = str(new_perc)
        journey_activities["activities"][0]["outcomes"][0]["metaData"]["label"] = str(new_perc) + "%"
        journey_activities["activities"][0]["outcomes"][1]["arguments"]["percentage"] = str(100-new_perc)
        journey_activities["activities"][0]["outcomes"][1]["metaData"]["label"] = str(100-new_perc) + "%"
        return journey_activities

In [24]:
test = JourneyBuilder(client_id, client_secret, "test_2910")

In [25]:
id, version, status = test.get_metadata("id", "version", "status")
id, version, status

('e1906618-b1e6-4649-a159-1e382cc483c6', 1, 'Published')

In [40]:
test.update_journey_version('e1906618-b1e6-4649-a159-1e382cc483c6', 1, 33)

Journey: ('e1906618-b1e6-4649-a159-1e382cc483c6', 1) PAUSED SUCCESSFULLY!
{'id': 'e1906618-b1e6-4649-a159-1e382cc483c6', 'key': 'efe39a6b-de1d-a04a-dd3f-e546e67150a1', 'name': 'test_2910', 'lastPublishedDate': '2021-10-29T03:44:48', 'description': '', 'version': 1, 'workflowApiVersion': 1.0, 'createdDate': '2021-10-29T03:44:30.693', 'modifiedDate': '2021-10-29T05:32:12.823', 'activities': [{'id': '78a50523-72ac-45db-8268-75a6f969c628', 'key': 'RANDOMSPLITV2-1', 'name': '', 'description': '', 'type': 'RANDOMSPLIT', 'outcomes': [{'key': 'branchResult-1', 'next': 'WAITBYDURATION-1', 'arguments': {'percentage': '75'}, 'metaData': {'label': '75%', 'pathName': 'Path 1', 'invalid': False}}, {'key': 'branchResult-2', 'next': 'WAITBYDURATION-2', 'arguments': {'percentage': '25'}, 'metaData': {'label': '25%', 'pathName': 'Path 2', 'invalid': False}}], 'arguments': {}, 'configurationArguments': {}, 'metaData': {'icon': '/extensions/activities/random-split/images/randomsplit.svg', 'iconSmall': '/e

Journey: ('e1906618-b1e6-4649-a159-1e382cc483c6', 1) RESUMED SUCCESSFULLY!
{'id': 'e1906618-b1e6-4649-a159-1e382cc483c6', 'key': 'efe39a6b-de1d-a04a-dd3f-e546e67150a1', 'name': 'test_2910', 'lastPublishedDate': '2021-10-29T03:44:48', 'description': '', 'version': 1, 'workflowApiVersion': 1.0, 'createdDate': '2021-10-29T03:44:30.693', 'modifiedDate': '2021-10-29T05:37:18.123', 'activities': [{'id': '5a787041-a97c-48af-bc04-51d678d5775f', 'key': 'RANDOMSPLITV2-1', 'name': '', 'description': '', 'type': 'RANDOMSPLIT', 'outcomes': [{'key': 'branchResult-1', 'next': 'WAITBYDURATION-1', 'arguments': {'percentage': '33'}, 'metaData': {'label': '33%', 'pathName': 'Path 1', 'invalid': False}}, {'key': 'branchResult-2', 'next': 'WAITBYDURATION-2', 'arguments': {'percentage': '67'}, 'metaData': {'label': '67%', 'pathName': 'Path 2', 'invalid': False}}], 'arguments': {}, 'configurationArguments': {}, 'metaData': {'icon': '/extensions/activities/random-split/images/randomsplit.svg', 'iconSmall': '/

In [13]:
# version still persists although deleted (version 2 deleted but next version created is 3)
# json.dumps(data) to make it a string - that was the key